In [19]:

# Weights and Biases related imports
import wandb
from wandb.keras import WandbMetricsLogger

In [20]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [21]:
wandb.login()

True

In [22]:
sweep_config = {
    'method': 'grid'
    }

metric = {
    'name': 'val_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'reg_alpha': {
          'values': [0] + np.logspace(-5, -2, num=3).tolist()
        },
    'del_5x5': {
          'values': [False, True]
        },
    'add_7x7': {
          'values': [False, True]
        }
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'earlystopping_patience': {
        'value': 10},
    'epochs': {
        'value': 100},
    'learning_rate': {
        'value': 0.000016},
    'batch_size': {
        'value': 128},
    })

In [23]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'add_7x7': {'values': [False, True]},
                'batch_size': {'value': 128},
                'del_5x5': {'values': [False, True]},
                'earlystopping_patience': {'value': 10},
                'epochs': {'value': 100},
                'learning_rate': {'value': 1.6e-05},
                'reg_alpha': {'values': [0,
                                         1e-05,
                                         0.00031622776601683794,
                                         0.01]}}}


In [24]:
sweep_id = 'takim/CIFAR-10_Classification/6m2el5jq'

In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, Lambda, Add, Input, GlobalAveragePooling2D, Flatten, Dense, Softmax, MaxPooling2D, AveragePooling2D, Dropout
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.regularizers import l2


def InceptionBlock(x, f1, f3_reduce, f3, f5_reduce, f5, pool_reduce, reg_alpha, del_5x5, add_7x7):

    f7_reduce = f5_reduce
    f7 = f5

    p1_x = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    p1_x = ReLU()(p1_x)

    p3_x = Conv2D(filters=f3_reduce, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    p3_x = ReLU()(p3_x)
    p3_x = Conv2D(filters=f3, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(p3_x)
    p3_x = ReLU()(p3_x)

    if not del_5x5:
        p5_x = Conv2D(filters=f5_reduce, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
        p5_x = ReLU()(p5_x)
        p5_x = Conv2D(filters=f5, kernel_size=(5, 5), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(p5_x)
        p5_x = ReLU()(p5_x)

    if add_7x7:
        p7_x = Conv2D(filters=f7_reduce, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
        p7_x = ReLU()(p7_x)
        p7_x = Conv2D(filters=f7, kernel_size=(7, 7), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(p7_x)
        p7_x = ReLU()(p7_x)  

    pool_x = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    pool_x = Conv2D(filters=pool_reduce, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(pool_x)
    pool_x = ReLU()(pool_x)

    if del_5x5 and add_7x7:
        x = concatenate(inputs=[p1_x, p3_x, p7_x, pool_x], axis=-1)
    elif del_5x5 and not add_7x7:
        x = concatenate(inputs=[p1_x, p3_x, pool_x], axis=-1)
    elif not del_5x5 and add_7x7:
        x = concatenate(inputs=[p1_x, p3_x, p5_x, p7_x, pool_x], axis=-1)
    elif not del_5x5 and not add_7x7:
        x = concatenate(inputs=[p1_x, p3_x, p5_x, pool_x], axis=-1)
        
    return x

def create_model(config):

    reg_alpha = config['reg_alpha']
    del_5x5 = config['del_5x5']
    add_7x7 = config['add_7x7']

    inputs = Input(shape=(32, 32, 3))

    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', kernel_regularizer=l2(reg_alpha))(inputs)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
            
    x = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    x = ReLU()(x)
    x = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = InceptionBlock(x, 64, 96, 128, 16, 32, 32, reg_alpha, del_5x5, add_7x7)
    x = InceptionBlock(x, 128, 128, 192, 32, 96, 64, reg_alpha, del_5x5, add_7x7)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = InceptionBlock(x, 192, 96, 208, 16, 48, 64, reg_alpha, del_5x5, add_7x7)

    # Auxilary loss-output
    x_aux1 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='same')(x)
    x_aux1 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x_aux1)
    x_aux1 = ReLU()(x_aux1)
    x_aux1 = Flatten()(x_aux1)
    x_aux1 = Dense(1024, kernel_regularizer=l2(reg_alpha))(x_aux1)
    x_aux1 = ReLU()(x_aux1)
    x_aux1 = Dropout(0.7)(x_aux1)
    x_aux1 = Dense(10)(x_aux1)
    x_aux1 = Softmax(name='aux1_out')(x_aux1)

    x = InceptionBlock(x, 160, 112, 224, 24, 64, 64, reg_alpha, del_5x5, add_7x7)
    x = InceptionBlock(x, 128, 128, 256, 24, 64, 64, reg_alpha, del_5x5, add_7x7)
    x = InceptionBlock(x, 112, 144, 288, 32, 64, 64, reg_alpha, del_5x5, add_7x7)

    # Auxilary loss-output
    x_aux2 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='same')(x)
    x_aux2 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg_alpha))(x_aux2)
    x_aux2 = ReLU()(x_aux2)
    x_aux2 = Flatten()(x_aux2)
    x_aux2 = Dense(1024, kernel_regularizer=l2(reg_alpha))(x_aux2)
    x_aux2 = ReLU()(x_aux2)
    x_aux2 = Dropout(0.7)(x_aux2)
    x_aux2 = Dense(10)(x_aux2)
    x_aux2 = Softmax(name='aux2_out')(x_aux2)

    x = InceptionBlock(x, 256, 160, 320, 32, 128, 128, reg_alpha, del_5x5, add_7x7)

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    x = InceptionBlock(x, 256, 160, 320, 32, 128, 128, reg_alpha, del_5x5, add_7x7)
    x = InceptionBlock(x, 384, 192, 384, 48, 128, 128, reg_alpha, del_5x5, add_7x7)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(10)(x)
    x = Softmax(name='og_out')(x)

    outputs = [x_aux1, x_aux2, x]

    model = Model(inputs, outputs, name='GoogLeNet')

    return model

In [26]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

def train(config = None):
    with wandb.init(config=config):

        config = wandb.config

        tf.keras.backend.clear_session()
        model = create_model(config)
        model.compile(
            loss=["categorical_crossentropy", "categorical_crossentropy", "categorical_crossentropy"],
            loss_weights=[0.3, 0.3, 1],
            metrics = ["accuracy", tf.keras.metrics.TopKCategoricalAccuracy(k=3, name='top@3_accuracy')]
        )

        early_stopping = EarlyStopping(monitor='val_og_out_loss',
                                    patience=config["earlystopping_patience"],
                                    restore_best_weights=True)

        history = model.fit(x_train, y_train,
                                    epochs=config["epochs"],
                                    batch_size=config["batch_size"],
                                    validation_split=0.1,
                                    callbacks=[
                                        WandbMetricsLogger(log_freq='epoch'),
                                        early_stopping
                                    ], verbose=1
                                    )
        
        test_stats = model.evaluate(x_test, y_test)
        wandb.log({"test_og_out_loss": test_stats[3]})
        wandb.log({"test_og_out_acc": test_stats[8]})

        val_loss_history = history.history['val_og_out_loss']
        val_acc_history = history.history['val_og_out_accuracy']

        best_epoch_num = -1 if (len(val_loss_history) == 100 or len(val_loss_history) <= 10) else (len(val_loss_history) - 11)

        wandb.log({"best_val_loss": val_loss_history[best_epoch_num]})
        wandb.log({"best_val_acc": val_acc_history[best_epoch_num]})

In [27]:
# wandb.agent(sweep_id, train, count=1)
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: lhtsl3gn with config:
wandb: 	add_7x7: True
wandb: 	batch_size: 128
wandb: 	del_5x5: False
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 1.6e-05
wandb: 	reg_alpha: 1e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
352/352 [==============================] - 35s 76ms/step - loss: 3.8543 - aux1_out_loss: 2.3808 - aux2_out_loss: 2.3506 - og_out_loss: 2.3934 - aux1_out_accuracy: 0.1106 - aux1_out_top@3_accuracy: 0.3254 - aux2_out_accuracy: 0.1074 - aux2_out_top@3_accuracy: 0.3201 - og_out_accuracy: 0.1080 - og_out_top@3_accuracy: 0.3226 - val_loss: 5.5974 - val_aux1_out_loss: 2.6445 - val_aux2_out_loss: 3.1007 - val_og_out_loss: 3.8573 - val_aux1_out_accuracy: 0.1064 - val_aux1_out_top@3_accuracy: 0.3064 - val_aux2_out_accuracy: 0.1064 - val_aux2_out_top@3_accuracy: 0.3026 - val_og_out_accuracy: 0.1064 - val_og_out_top@3_accuracy: 0.3064
Epoch 2/100
352/352 [==============================] - 25s 71ms/step - loss: 3.7558 - aux1_out_loss: 2.3565 - aux2_out_loss: 2.3353 - og_out_loss: 2.3390 - aux1_out_accuracy: 0.1228 - aux1_out_top@3_accuracy: 0.3636 - aux2_out_accuracy: 0.1224 - aux2_out_top@3_accuracy: 0.3613 - og_out_accuracy: 0.1238 - og_out_top@3_accuracy: 0.3654 - val_loss: 3.4409 - 

best_val_acc,▁
best_val_loss,▁
epoch/aux1_out_accuracy,▁▁▂▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇██████████
epoch/aux1_out_loss,██▇▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch/aux1_out_top@3_accuracy,▁▁▃▅▆▆▇▇▇▇▇▇██████████████████
epoch/aux2_out_accuracy,▁▁▂▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇██████████
epoch/aux2_out_loss,██▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch/aux2_out_top@3_accuracy,▁▁▃▄▆▆▆▇▇▇▇▇▇▇████████████████
epoch/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,██▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: eaui4zpq with config:
wandb: 	add_7x7: True
wandb: 	batch_size: 128
wandb: 	del_5x5: False
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 1.6e-05
wandb: 	reg_alpha: 0.00031622776601683794
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
352/352 [==============================] - 30s 71ms/step - loss: 4.4693 - aux1_out_loss: 2.4027 - aux2_out_loss: 2.3737 - og_out_loss: 2.4691 - aux1_out_accuracy: 0.1153 - aux1_out_top@3_accuracy: 0.3359 - aux2_out_accuracy: 0.1069 - aux2_out_top@3_accuracy: 0.3235 - og_out_accuracy: 0.1096 - og_out_top@3_accuracy: 0.3251 - val_loss: 3.7102 - val_aux1_out_loss: 2.2888 - val_aux2_out_loss: 2.2868 - val_og_out_loss: 2.2850 - val_aux1_out_accuracy: 0.1024 - val_aux1_out_top@3_accuracy: 0.3084 - val_aux2_out_accuracy: 0.0948 - val_aux2_out_top@3_accuracy: 0.3082 - val_og_out_accuracy: 0.0902 - val_og_out_top@3_accuracy: 0.3114
Epoch 2/100
352/352 [==============================] - 24s 69ms/step - loss: 3.6452 - aux1_out_loss: 2.2491 - aux2_out_loss: 2.2552 - og_out_loss: 2.2643 - aux1_out_accuracy: 0.1335 - aux1_out_top@3_accuracy: 0.3943 - aux2_out_accuracy: 0.1297 - aux2_out_top@3_accuracy: 0.3838 - og_out_accuracy: 0.1274 - og_out_top@3_accuracy: 0.3821 - val_loss: 3.3104 - 

...

best_val_acc,▁
best_val_loss,▁
epoch/aux1_out_accuracy,▁▁▁▂▂▂▃▃▃▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██████████
epoch/aux1_out_loss,███▇▇▆▆▅▅▅▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
epoch/aux1_out_top@3_accuracy,▁▁▁▂▃▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████████████
epoch/aux2_out_accuracy,▁▁▁▂▂▂▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████
epoch/aux2_out_loss,████▇▆▆▆▅▅▅▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch/aux2_out_top@3_accuracy,▁▁▁▃▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▇▆▆▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁


wandb: Agent Starting Run: 0s7iynxh with config:
wandb: 	add_7x7: True
wandb: 	batch_size: 128
wandb: 	del_5x5: True
wandb: 	earlystopping_patience: 10
wandb: 	epochs: 100
wandb: 	learning_rate: 1.6e-05
wandb: 	reg_alpha: 0.01
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/100
352/352 [==============================] - 24s 57ms/step - loss: 10.1683 - aux1_out_loss: 2.3379 - aux2_out_loss: 2.3057 - og_out_loss: 2.3156 - aux1_out_accuracy: 0.1024 - aux1_out_top@3_accuracy: 0.3098 - aux2_out_accuracy: 0.0962 - aux2_out_top@3_accuracy: 0.2961 - og_out_accuracy: 0.0969 - og_out_top@3_accuracy: 0.2953 - val_loss: 3.7012 - val_aux1_out_loss: 2.3026 - val_aux2_out_loss: 2.3026 - val_og_out_loss: 2.3026 - val_aux1_out_accuracy: 0.1064 - val_aux1_out_top@3_accuracy: 0.3058 - val_aux2_out_accuracy: 0.1064 - val_aux2_out_top@3_accuracy: 0.2992 - val_og_out_accuracy: 0.0958 - val_og_out_top@3_accuracy: 0.2966
Epoch 2/100
352/352 [==============================] - 19s 55ms/step - loss: 3.7014 - aux1_out_loss: 2.3029 - aux2_out_loss: 2.3028 - og_out_loss: 2.3027 - aux1_out_accuracy: 0.1002 - aux1_out_top@3_accuracy: 0.3006 - aux2_out_accuracy: 0.1000 - aux2_out_top@3_accuracy: 0.2996 - og_out_accuracy: 0.0999 - og_out_top@3_accuracy: 0.2960 - val_loss: 3.7012 -

best_val_acc,▁
best_val_loss,▁
epoch/aux1_out_accuracy,█▅▅▆▃▃▅▁▃▃▄▄▄
epoch/aux1_out_loss,█▁▁▁▁▁▁▁▁▁▁▁▁
epoch/aux1_out_top@3_accuracy,█▄▂▄▂▂▂▂▂▃▃▃▁
epoch/aux2_out_accuracy,▁▇█▆▆▇▅▄▄▅▅█▆
epoch/aux2_out_loss,█▁▁▁▁▁▁▁▁▁▁▁▁
epoch/aux2_out_top@3_accuracy,▂█▄▅▅▄▃▇▆▃▃▅▁
epoch/epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
